In [1]:
%matplotlib qt
# NOTE: Running on non-linear-axis Hyperspy branch
import hyperspy.api as hs
import pyxem as pxm
import numpy as np
import matplotlib.pyplot as plt
plt.style.use(['seaborn-talk'])
import os
import pickle
import addcopyfighandler


# XRD peak decay processing

In [39]:
dps = hs.load('data/diffraction_decay.hspy')
dps.plot()

## Select peak of interest (ROI)
Crop normalised data to small range containing the peak of interest. 

In [40]:
q_of_interest = 1.082
integration_range = 0.01

### Peak integration
Integrate the ROI:

In [42]:
dpc = dps.isig[(q_of_interest - integration_range):(q_of_interest + integration_range)]
dpc.plot()

In [43]:
integral = dpc.integrate1D(axis=1)
integral.plot()
plt.suptitle(y=0.95, x=0.6, t=('q_of_interest: ', q_of_interest, '; integration_range: ', integration_range))

Text(0.6, 0.95, "('q_of_interest: ', 1.082, '; integration_range: ', 0.01)")

# Intensity decay model fitting
Load all required files:

In [ ]:
integral


## a) Using `scipy` fitting

Fitting an exponential with `scipy` for the intensity decay.

The model to be fitted has the following expression:

$f(x)= A e^{-\frac{x}{\tau}}$
with parameters:
- $A:$ Maximum intensity
- $\tau:$ Scale parameter (time constant)

In [65]:
# Create xy array
x = np.vstack(dps.axes_manager.navigation_axes[0].axis)
x_units = dps.axes_manager.navigation_axes[0].units
y = np.vstack(integral.data)

xy = np.hstack((x,y))
xy

array([[0.00000000e+00, 4.12589388e-01],
       [1.00000000e+00, 4.08486998e-01],
       [2.00000000e+00, 4.20305356e-01],
       ...,
       [5.97000000e+02, 9.82420728e-02],
       [5.98000000e+02, 9.76431654e-02],
       [5.99000000e+02, 9.98495650e-02]])

In [66]:
# Remove points where NaN values as y
xy = xy[~np.isnan(xy)[:,1]]

In [70]:
from scipy.optimize import curve_fit
def exp_func(x, A, tau):
    return A*np.exp(-x / tau)

param, param_cov = curve_fit(exp_func, xy[:,0], xy[:,1], )
param_std = np.sqrt(np.diag(param_cov))

In [71]:
exp_fit = exp_func(xy[:,0], param[0], param[1])

plt.plot(xy[:,0], xy[:,1], label='Data')
plt.plot(xy[:,0], exp_fit, 'C2',
         label=('Fit: $({A:.2f} \pm {A_std:.2f}) \cdot exp[-x / ({tau:.2f} \pm {tau_std:.2f})]$'.format(A = param[0], A_std = param_std[0], tau = param[1], tau_std = param_std[1])))
plt.xlabel('Flux ({})'.format(x_units))
plt.legend()
plt.suptitle("Peak decay", fontsize='xx-large')



Text(0.5, 0.98, 'Peak decay')